In [2]:
%matplotlib ipympl

import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz
from scipy.signal import welch, periodogram
import physoce.tseries as ts
import gsw

In [3]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

In [4]:
ds

<xarray.Dataset>
Dimensions:    (beam: 4, bin: 23, time: 98208)
Coordinates:
  * time       (time) datetime64[ns] 2017-11-10T10:03:13.760000 ... 2018-10-17T09:58:13.720000
Dimensions without coordinates: beam, bin
Data variables:
    Pit        (time) float64 ...
    Rol        (time) float64 ...
    Hea        (time) float64 ...
    Tem        (time) float64 ...
    Dep        (time) float64 ...
    Ori        (time) int64 ...
    BIT        (time) int64 ...
    Bat        (time) int64 ...
    Eas        (time, bin) float64 ...
    Nor        (time, bin) float64 ...
    Ver        (time, bin) float64 ...
    Err        (time, bin) float64 ...
    EA         (time, bin, beam) float64 ...
    PG         (time, bin, beam) float64 ...
    C          (time, bin, beam) float64 ...
    binheight  (bin) float64 ...
    Eas_tide   (time, bin) float64 ...
    Nor_tide   (time, bin) float64 ...
    Eas_filt   (time, bin) float64 ...
    Nor_filt   (time, bin) float64 ...
Attributes:
    PingsPer

In [5]:
ds['time']

<xarray.DataArray 'time' (time: 98208)>
array(['2017-11-10T10:03:13.760000000', '2017-11-10T10:08:13.760000000',
       '2017-11-10T10:13:13.760000000', ..., '2018-10-17T09:48:13.720000000',
       '2018-10-17T09:53:13.720000000', '2018-10-17T09:58:13.720000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2017-11-10T10:03:13.760000 ... 2018-10-17T09:58:13.720000

In [6]:
T_f_hrs = 1/(gsw.f(35+8.4585/60)/(2*np.pi))/3600

Nor_bpf = ts.pl64(np.array(ds['Nor']),dt=1/12,T=T_f_hrs/0.95) - ts.pl64(np.array(ds['Nor']),dt=1/12,T=T_f_hrs/1.15)
ds['Nor_bpf'] = (('time','bin'),Nor_bpf)
Eas_bpf = ts.pl64(np.array(ds['Eas']),dt=1/12,T=T_f_hrs/0.95) - ts.pl64(np.array(ds['Eas']),dt=1/12,T=T_f_hrs/1.15) 
ds['Eas_bpf'] = (('time','bin'),Eas_bpf)

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


In [89]:
plt.figure()
plt.plot(np.mean(ds['C'],axis=0),ds['binheight'],'-o')
plt.title('correlation [128 = high signal/noise]')
plt.ylabel('height above instrument [m]')
plt.xlim([0,128])

plt.figure()
plt.plot(np.std(ds['Eas'],axis=0),ds['binheight'],'-o')
plt.plot(np.std(ds['Nor'],axis=0),ds['binheight'],'-o')
plt.title('std(u,v)')
plt.ylabel('height above instrument [m]')

plt.figure()
plt.plot(np.mean(ds['EA'],axis=0),ds['binheight'],'-o')
plt.title('echo amplitude [dB]')
plt.ylabel('height above instrument [m]')

plt.figure()
plt.plot(np.corrcoef(ds['Eas'],rowvar=False)[:,0],ds['binheight'],'-o')
plt.plot(np.corrcoef(ds['Nor'],rowvar=False)[:,0],ds['binheight'],'-o')
plt.title('correlation with bottom bin')
plt.ylabel('height above instrument [m]')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0, 0.5, 'height above instrument [m]')

In [90]:
bini = np.arange(1,17)

plt.figure()
plt.plot(np.mean(ds['Eas'][:,bini],axis=0),ds['binheight'][bini],'-o')
plt.plot(np.mean(ds['Nor'][:,bini],axis=0),ds['binheight'][bini],'-o')
plt.title('mean velocity - entire deployment')
plt.legend(['East','North'])
plt.ylabel('height above instrument')
plt.xlabel('[m/s]')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0.5, 0, '[m/s]')

In [11]:
dt = 5*60 # 5 min sample interval (in seconds)

x = cumtrapz(ds['Eas']*dt,axis=0)
y = cumtrapz(ds['Nor']*dt,axis=0)

In [12]:
plt.figure()
plt.plot(x[:,15]/1000,y[:,15]/1000)
plt.plot(x[:,7]/1000,y[:,8]/1000)
plt.plot(x[:,1]/1000,y[:,1]/1000)
plt.plot(0,0,'ko')
plt.axis('equal')
plt.xlabel('distance east [km]')
plt.ylabel('distance north [km]')
plt.legend(['Bin 15','Bin 8','Bin 1'])

FigureCanvasNbAgg()

In [13]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas'][:,15]);
plt.plot(ds['time'],ds['Eas'][:,1]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('eastward velocity')

plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Nor'][:,15]);
plt.plot(ds['time'],ds['Nor'][:,1]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('northward velocity')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0.5, 1.0, 'northward velocity')

In [14]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas_filt'][:,15]);
plt.plot(ds['time'],ds['Eas_filt'][:,1]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('eastward velocity')

plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Nor_filt'][:,15]);
plt.plot(ds['time'],ds['Nor_filt'][:,1]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('northward velocity')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0.5, 1.0, 'northward velocity')

In [15]:
from graph import stick_plot



fig, ax = plt.subplots(figsize=(8, 2.25))

q = stick_plot(np.array(ds['time'])[::288],
               np.array(ds['Eas_filt'])[::288,15],
               np.array(ds['Nor_filt'])[::288,15],
               ax=ax,width=0.0015)

ref = 0.01
qk = plt.quiverkey(q, 0.1, 0.85, ref,
                  "%s m s$^{-1}$" % ref,
                  labelpos='N', coordinates='axes')
_ = plt.xticks(rotation=0)

plt.title('Station M - low pass filtered velocity - bin 15')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'Station M - low pass filtered velocity - bin 15')

In [16]:
t1=np.datetime64('2018-02-01')
t2=np.datetime64('2018-02-15')

plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas'][:,15]);
plt.plot(ds['time'],ds['Nor'][:,15]);
plt.ylabel('m/s')
plt.xlim([t1,t2])
plt.ylim([-0.15,0.15])
plt.legend(['East','North'])
plt.title('full velocity, '+str(t1)+'-'+str(t2)+', bin 15')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'full velocity, 2018-02-01-2018-02-15, bin 15')

In [17]:
t1=np.datetime64('2018-02-01')
t2=np.datetime64('2018-02-15')

plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas'][:,15]-ds['Eas_tide'][:,15]);
plt.plot(ds['time'],ds['Nor'][:,15]-ds['Nor_tide'][:,15]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.ylim([-0.15,0.15])
plt.xlim([t1,t2])
plt.legend(['East','North'])
plt.title('detided velocity, '+str(t1)+'-'+str(t2)+', bin 15')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'detided velocity, 2018-02-01-2018-02-15, bin 15')

In [18]:
Eas = np.array(ds['Eas'])
Nor = np.array(ds['Nor'])
Eas_tide = np.array(ds['Eas_tide'])
Nor_tide = np.array(ds['Nor_tide'])

N = len(Eas)
fE,SE = welch(Eas[:,5],fs=288.,nperseg=N/6,window='hanning')
fN,SN = welch(Nor[:,5],fs=288.,nperseg=N/6,window='hanning')

#rotary spectrum
fr,Sr = welch(Eas[:,0]+1j*Nor[:,0],
              fs=288.,nperseg=N/6,
              window='hann',return_onesided=False)
icw, = np.where(fr<0)
iccw, = np.where(fr>0)
fcw = -fr[icw]
Scw = Sr[icw]
fccw = fr[iccw]
Sccw = Sr[iccw]

In [19]:
fcw[1:]

array([1.43982405e+02, 1.43964809e+02, 1.43947214e+02, ...,
       5.27859238e-02, 3.51906158e-02, 1.75953079e-02])

In [20]:
fccw[::-1]

array([1.43982405e+02, 1.43964809e+02, 1.43947214e+02, ...,
       5.27859238e-02, 3.51906158e-02, 1.75953079e-02])

In [21]:
# rotary coefficient
Cr = np.abs(Scw[1:]-Sccw[::-1])/(Scw[1:]+Sccw[::-1])
fr = fcw[1:]

In [22]:
f_cpd = gsw.f(35+8.4585/60)*86400/(2*np.pi)

# theoretical prediction (Van Haren and Millot (2004), eq. 11)
sigmar = 2*np.pi*fr
Cr_pred = 2*sigmar*(2*np.pi*f_cpd)/(sigmar**2 + (2*np.pi*f_cpd)**2)

In [23]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('rotary spectra')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'rotary spectra')

In [24]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')

yl = plt.ylim()
plt.plot([f_cpd,f_cpd],yl,'-',color='gray')
plt.plot([24/25.82,24/25.82],yl,'-',color='gray')
plt.plot([24/23.93,24/23.93],yl,'-',color='gray')
plt.plot([2,2],yl,'-',color='gray')
plt.plot([24/12.42,24/12.42],yl,'-',color='gray')
plt.plot([24/12.42*2,24/12.42*2],yl,'-',color='gray')
plt.ylim(yl)
plt.xlim([0.6,10])

plt.text(f_cpd,yl[1],'$f$',fontsize=12,
         horizontalalignment='center',
         verticalalignment='bottom')
plt.text(24/12.42,yl[1],'$M_2$',fontsize=12,
         horizontalalignment='right',
         verticalalignment='bottom')
plt.text(24/12.42*2,yl[1],'$2M_2$',fontsize=12,
         horizontalalignment='center',
         verticalalignment='bottom')
plt.text(2,yl[1],'$S_2$',fontsize=12,
         horizontalalignment='left',
         verticalalignment='bottom')
plt.text(24/23.93,yl[1],'$K_1$',fontsize=12,
         horizontalalignment='center',
         verticalalignment='bottom')
plt.text(24/25.82,yl[1],'$O_1$',fontsize=12,
         horizontalalignment='right',
         verticalalignment='bottom')

FigureCanvasNbAgg()

Text(0.9295120061967467, 0.01597311802948467, '$O_1$')

In [25]:
iwi, = np.where(fr >= f_cpd)

plt.figure()
plt.semilogx(fr,Cr)
plt.semilogx(fr[iwi],Cr_pred[iwi],'k-',lw=2)
yl = plt.ylim()
plt.semilogx([f_cpd,f_cpd],yl,'k--')
plt.xlabel('f [cpd]')
plt.ylabel('$C_R$')
plt.title('rotary coefficients')
plt.ylim(yl)
plt.xlim([0.6,10])
plt.ylim([0,1])

FigureCanvasNbAgg()

(0, 1)

In [26]:
plt.figure()
plt.plot(ds['Eas_bpf'][:,10],ds['Nor_bpf'][:,10])
plt.gca().axis('equal')

FigureCanvasNbAgg()

(-0.01632384540970697,
 0.016935338900897837,
 -0.015948528018772588,
 0.015739624961456644)

In [27]:
plt.figure()
plt.plot(ds['time'],ds['Eas_bpf'][:,15])
plt.plot(ds['time'],ds['Nor_bpf'][:,15])

FigureCanvasNbAgg()

In [28]:
plt.figure()
plt.plot(ds['time'],ds['Eas_bpf'][:,15])
plt.plot(ds['time'],ds['Eas_bpf'][:,0])

FigureCanvasNbAgg()

In [ ]:
ds['height'] = ds['binheight'] + 0.4572 # add instrument height (18 inches)
zi = np.arange(0,len(ds['height'])-7)

w = Eas[:,zi]+1j*Nor[:,zi]

In [ ]:
np.cov(w,rowvars=False)